<a href="https://colab.research.google.com/github/duytruong204/Project/blob/main/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
#my code
## elements là mảng chứa thông tin về tất cả các button và block (có thể có logo nữa: tạm thời chưa xét logo)
# mỗi phần tử của mảng gồm 5 thành phần tương ứng với các index:
#           - index 0: thể hiện loại của đối tượng: 0 là button, 1 là logo, 2 là block
#           - index 1: chiều rộng đối tượng; index 2: chiều cao đối tượng
#           - index 3: mảng chứa các màu có trong button với độ bao phủ kèm theo
#           - index 4: bán kính bo góc của phần tử (là số thực từ 0 đến 0.5: từ vuông đến tròn hoàn toàn)
## colors là mảng chứa tất cả các màu với % độ bao phủ, trừ màu neutral và màu trong real image
## neutralColors là mảng chứa tất cả các màu neutral với % độ bao phủ
## chỉ lấy button
!pip install colour

#Library

In [65]:
import pandas as pd
from colour import Color
import math
import time
import numpy as np

#Function

In [66]:
def to_list(mydataframe):
  color_of_button = mydataframe.iloc[:, [1]].values 
  list_of_button = []
  list_color = []
  for i in range(color_of_button.shape[0]): 
    list_of_button.append(color_of_button[i][0])
  for i in range(len(list_of_button)):
    for j in range(len(list_of_button[i])):
      list_color.append(list_of_button[i][j])
  return list_color

In [67]:
def turn_none(element):
  if element is None:
    return []
  else:
    return element

In [68]:
def make_second(list1,list2,list3):
  sum = []
  a = []
  sum = turn_none(list1) + turn_none(list2) + turn_none(list3)
  for i in range(len(sum)):
    a.append(sum[i][0])
  return a

In [69]:
def soft_coverage(list):
  if list is None:
    return None
  else:
    data_list = pd.DataFrame(list, columns=['Color','coverage'])
    data_list = data_list.sort_values(by=['coverage'],ascending=False)#sort
    return coverage_to_list(data_list)

In [70]:
def delete_element(list1,list2):
  a = []
  b = []
  if list2 is None:
    return list1
  else:
    for i in range(len(list2)):
      b.append(list2[i][0])
    for i in range(len(list1)):
      if (list1[i][0] in b):
        a.append(list1[i]);
  if len(a) == 0:
    return None
  return a

In [71]:
def rest_element(list1,list2):
  a = []
  b = []
  if list2 is None:
    return None
  elif list1 is None:
    return list2 
  else:
    for i in range(len(list1)):
      b.append(list1[i][0])
    for i in range(len(list2)):
      if not list2[i][0] in b:
        a.append(list2[i]);
  if len(a) == 0:
    return None
  return a 

In [72]:
rest_element([(2,0)],[(3,0),(4,0),(2,0)])

[(3, 0), (4, 0)]

In [73]:
def coverage_to_list(mydataframe):
  list_color = []
  for i in range(mydataframe.shape[0]):
    list_color.append([mydataframe.iloc[:, [0,1]].values[i][0],mydataframe.iloc[:, [0,1]].values[i][1]])
  return list_color

In [74]:
def palette_type(list_primary,list_secondary):
  if list_primary == [[]]:
    return ['nothing']
  elif len(list_primary) == 1 and len(list_secondary[0][0:5]) == 0:
    return ['Monotune']
  elif len(list_primary) == 1 and len(list_secondary[0][0:5]) == 1:
    return ['Duetune']
  elif len(list_primary) == 1 and len(list_secondary[0][0:5]) >= 2:
    return ['Colorful']

In [75]:
def main_take_color(elements,colors,data_list):
  list_primary = []
  list_secondary = []
  ##take primary color
  new_element = pd.DataFrame(elements, columns=['Type', 'Width ', 'Height','Color of Button',"Radius"])
  new_element['Area'] = (new_element.iloc[:, [1]].values * new_element.iloc[:, [2]].values )
  just_button = new_element[['Type', 'Color of Button','Area']].loc[new_element['Type'] == 0]#get button
  just_biggest_button = just_button.loc[just_button['Area'] == just_button['Area'].max()]#getmax
  list_color_of_button = pd.DataFrame(to_list(just_biggest_button), columns=['Color', 'coverage'])
  list_color_of_button = list_color_of_button.loc[list_color_of_button['coverage'] == list_color_of_button['coverage'].max()]
  color_coverage = pd.DataFrame(colors,columns=['Color',"coverage"])
  color_coverage = color_coverage.loc[color_coverage['coverage'] == color_coverage['coverage'].max()]
  if list_color_of_button.shape[0] == 1:
    primary = list_color_of_button.iloc[[0],[0]].values[0][0] 
    list_primary.append(list_color_of_button.iloc[[0],[0]].values[0][0])
  elif color_coverage.shape[0] == 1:
    primary = color_coverage.iloc[[0],[0]].values[0][0]
    list_primary.append(color_coverage.iloc[[0],[0]].values[0][0])
  else:
    primary = None
    list_primary = [[]]
  ##take secondary
  #list of button
  list_color_of_button = pd.DataFrame(to_list(just_button), columns=['Color', 'coverage'])
  list_color_of_button = list_color_of_button.loc[list_color_of_button['Color'] != primary]#exclude primary
  list_color_of_button = list_color_of_button.sort_values(by=['coverage'],ascending=False)#sort
  #color
  color_coverage = pd.DataFrame(colors,columns=['Color',"coverage"])
  color = color_coverage.loc[color_coverage['Color'] != primary]#exclude primary
  color = color.sort_values(by=['coverage'],ascending=False)#sort
  color_list = delete_element(coverage_to_list(color),coverage_to_list(list_color_of_button))
  color_logo = None
  rest_of_color = rest_element(color_list,coverage_to_list(color))
  list_secondary.append(make_second(soft_coverage(color_list),soft_coverage(color_logo),soft_coverage(rest_of_color)))
  ##get palette
  data_list.append(tuple(list_primary+[list_secondary[0][0:5]]+palette_type(list_primary,list_secondary)))

#**MAIN**

In [76]:
data_list = []#we're gonna store data on this

In [77]:
elements = [(0,
  152,
  32,
  [((27, 91, 61), 0.8577302631579029),
   ((33, 100, 72), 0.01562500000000001),
   ((42, 100, 90), 0.001850328947368421),
   ((21, 96, 81), 0.00041118421052631577)],
  'None'),
 (0,
  26,
  25,
  [((28, 71, 61), 0.8384615384615283),
   ((34, 100, 79), 0.03846153846153845),
   ((22, 66, 61), 0.007692307692307693),
   ((42, 100, 83), 0.007692307692307693)],
  'None')]
colors = [((27, 91, 61), 0.011777083333332926),
 ((193, 100, 88), 0.0006104166666666641),
 ((33, 100, 72), 0.0004729166666666653),
 ((58, 84, 68), 0.0009333333333333288),
 ((17, 81, 70), 0.00016874999999999976),
 ((197, 82, 87), 0.00021874999999999954),
 ((63, 79, 67), 7.500000000000002e-05),
 ((49, 82, 78), 0.00019374999999999964),
 ((9, 89, 86), 4.583333333333335e-05),
 ((5, 92, 90), 6.25e-06),
 ((181, 100, 91), 2.0833333333333334e-06)]

In [78]:
t = time.time()
main_take_color(elements,colors,data_list)
main_data = pd.DataFrame(data_list,columns=['Primary','Secondary','Palette type'])
print(f'Runtime: {time.time()-t}')

Runtime: 0.06324553489685059


In [79]:
#data color of domain
main_data.head()

,Primary,Secondary,Palette type
0,"(27, 91, 61)","[(33, 100, 72), (58, 84, 68), (193, 100, 88), ...",Colorful


#Take primary color

##Turn to dataframe and take only the biggest button



In [80]:
list_primary = []
list_secondary = []
new_element = pd.DataFrame(elements, columns=['Type', 'Width ', 'Height','Color of Button',"Radius"])
width = new_element.iloc[:, [1]].values  
height = new_element.iloc[:, [2]].values  
area = (width*height)
new_element['Area'] = area
just_button = new_element[['Type', 'Color of Button','Area']].loc[new_element['Type'] == 0]
just_button.head()

,Type,Color of Button,Area
0,0,"[((27, 91, 61), 0.8577302631579029), ((33, 100...",4864
1,0,"[((28, 71, 61), 0.8384615384615283), ((34, 100...",650


In [81]:
just_biggest_button = just_button.loc[just_button['Area'] == just_button['Area'].max()]#getmax
just_biggest_button.head()

,Type,Color of Button,Area
0,0,"[((27, 91, 61), 0.8577302631579029), ((33, 100...",4864


##When there're more than 2 color with the same size

In [82]:
color_coverage = pd.DataFrame(colors,columns=['Color',"coverage"])
color_coverage = color_coverage.loc[color_coverage['coverage'] == color_coverage['coverage'].max()]
color_coverage 

,Color,coverage
0,"(27, 91, 61)",0.011777


##Can't take primary color by size or % coverage

In [83]:
list_color_of_button = pd.DataFrame(to_list(just_biggest_button), columns=['Color', 'coverage'])
list_color_of_button = list_color_of_button.loc[list_color_of_button['coverage'] == list_color_of_button['coverage'].max()]
color_coverage = pd.DataFrame(colors,columns=['Color',"coverage"])
color_coverage = color_coverage.loc[color_coverage['coverage'] == color_coverage['coverage'].max()]
if list_color_of_button.shape[0] == 1:
  primary = list_color_of_button.iloc[[0],[0]].values[0][0] 
  list_primary.append(list_color_of_button.iloc[[0],[0]].values[0][0])
elif color_coverage.shape[0] == 1:
  primary = color_coverage.iloc[[0],[0]].values[0][0]
  list_primary.append(color_coverage.iloc[[0],[0]].values[0][0])
else:
  primary = None
  list_primary = [[]]
list_primary

[(27, 91, 61)]

In [84]:
primary

(27, 91, 61)

In [85]:
to_list(just_button)

[((27, 91, 61), 0.8577302631579029),
 ((33, 100, 72), 0.01562500000000001),
 ((42, 100, 90), 0.001850328947368421),
 ((21, 96, 81), 0.00041118421052631577),
 ((28, 71, 61), 0.8384615384615283),
 ((34, 100, 79), 0.03846153846153845),
 ((22, 66, 61), 0.007692307692307693),
 ((42, 100, 83), 0.007692307692307693)]

#Take secondary

###Take secondary color according to color used for button (exclude primary color)(###)



In [86]:
#list of button
list_color_of_button = pd.DataFrame(to_list(just_button), columns=['Color', 'coverage'])
list_color_of_button = list_color_of_button.loc[list_color_of_button['Color'] != primary]#exclude primary
list_color_of_button = list_color_of_button.sort_values(by=['coverage'],ascending=False)#sort
#color
color_coverage = pd.DataFrame(colors,columns=['Color',"coverage"])
color = color_coverage.loc[color_coverage['Color'] != primary]#exclude primary
color = color.sort_values(by=['coverage'],ascending=False)#sort

In [87]:
primary

(27, 91, 61)

In [88]:
coverage_to_list(list_color_of_button)

[[(28, 71, 61), 0.8384615384615283],
 [(34, 100, 79), 0.03846153846153845],
 [(33, 100, 72), 0.01562500000000001],
 [(22, 66, 61), 0.007692307692307693],
 [(42, 100, 83), 0.007692307692307693],
 [(42, 100, 90), 0.001850328947368421],
 [(21, 96, 81), 0.00041118421052631577]]

In [89]:
coverage_to_list(list_color_of_button)
color_list = delete_element(coverage_to_list(color),coverage_to_list(list_color_of_button))
print(color_list)

[[(33, 100, 72), 0.0004729166666666653]]


##Take secondary color according to Colors used in logo *(not now)

In [90]:
color_logo = None

##Take secondary colors according to % coverage in the shortlist

In [91]:
#Take rest of color
rest_of_color = rest_element(color_list,coverage_to_list(color))
rest_of_color

[[(58, 84, 68), 0.0009333333333333288],
 [(193, 100, 88), 0.0006104166666666641],
 [(197, 82, 87), 0.00021874999999999954],
 [(49, 82, 78), 0.00019374999999999964],
 [(17, 81, 70), 0.00016874999999999976],
 [(63, 79, 67), 7.500000000000002e-05],
 [(9, 89, 86), 4.583333333333335e-05],
 [(5, 92, 90), 6.25e-06],
 [(181, 100, 91), 2.0833333333333334e-06]]

In [92]:
list_secondary.append(make_second(soft_coverage(color_list),soft_coverage(color_logo),soft_coverage(rest_of_color)))
list_secondary

[[(33, 100, 72),
  (58, 84, 68),
  (193, 100, 88),
  (197, 82, 87),
  (49, 82, 78),
  (17, 81, 70),
  (63, 79, 67),
  (9, 89, 86),
  (5, 92, 90),
  (181, 100, 91)]]